<a href="https://colab.research.google.com/github/Nuamaan/NetFlix-Recommender/blob/main/IDEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn==0.22.2 --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 3.7 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2 which is incompatible.


In [ ]:
import os
import sys
import time
import math
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.data as data
import sklearn
import scipy
import collections
from scipy.linalg import norm
from PIL import Image


def weights_xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight.data)
        nn.init.constant(m.bias.data, 0)

def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    #from scipy.optimize import linear_sum_assignment as linear_assignment
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

class Dataset(data.Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.data = data
        self.labels = labels
        if torch.cuda.is_available():
            self.data = self.data.cuda()
            self.labels = self.labels.cuda()

    def __getitem__(self, index):
        img, target = self.data[index], self.labels[index]
        # img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import math
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cluster import KMeans


def buildNetwork(layers, activation="relu", dropout=0):
    net = []
    for i in range(1, len(layers)):
        net.append(nn.Linear(layers[i-1], layers[i]))
        if activation=="relu":
            net.append(nn.ReLU())
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if dropout > 0:
            net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)



class IDEC(nn.Module):
    def __init__(self, input_dim=784, z_dim=10, n_clusters=10,
        encodeLayer=[400], decodeLayer=[400], activation="relu", dropout=0, alpha=1., gamma=0.1):
        super(self.__class__, self).__init__()
        self.z_dim = z_dim
        self.layers = [input_dim] + encodeLayer + [z_dim]
        self.activation = activation
        self.dropout = dropout
        self.encoder = buildNetwork([input_dim] + encodeLayer, activation=activation, dropout=dropout)
        self.decoder = buildNetwork([z_dim] + decodeLayer, activation=activation, dropout=dropout)
        self._enc_mu = nn.Linear(encodeLayer[-1], z_dim)
        self._dec = nn.Linear(decodeLayer[-1], input_dim)

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.gamma = gamma
        self.mu = Parameter(torch.Tensor(n_clusters, z_dim))

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict) 
        self.load_state_dict(model_dict)

    def forward(self, x):
        h = self.encoder(x)
        z = self._enc_mu(h)
        h = self.decoder(z)
        xrecon = self._dec(h)
        # compute q -> NxK
        q = self.soft_assign(z)
        return z, q, xrecon

    def soft_assign(self, z):
        q = 1.0 / (1.0 + torch.sum((z.unsqueeze(1) - self.mu)**2, dim=2) / self.alpha)
        q = q**(self.alpha+1.0)/2.0
        q = q / torch.sum(q, dim=1, keepdim=True)
        return q

    def encodeBatch(self, X, batch_size=256):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        
        encoded = []
        self.eval()
        num = X.shape[0]
        num_batch = int(math.ceil(1.0*X.shape[0]/batch_size))
        for batch_idx in range(num_batch):
            xbatch = X[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
            inputs = Variable(xbatch)
            z,_, _ = self.forward(inputs)
            encoded.append(z.data)

        encoded = torch.cat(encoded, dim=0)
        return encoded

    def cluster_loss(self, p, q):
        def kld(target, pred):
            return torch.mean(torch.sum(target*torch.log(target/(pred+1e-6)), dim=1))
        kldloss = kld(p, q)
        return self.gamma*kldloss

    def recon_loss(self, x, xrecon):
        recon_loss = torch.mean((xrecon-x)**2)
        return recon_loss

    def pairwise_loss(self, p1, p2, cons_type):
        if cons_type == "ML":
            ml_loss = torch.mean(-torch.log(torch.sum(p1 * p2, dim=1)))
            return ml_loss
        else:
            cl_loss = torch.mean(-torch.log(1.0 - torch.sum(p1 * p2, dim=1)))
            return cl_loss

    def global_size_loss(self, p, cons_detail):
        m_p = torch.mean(p, dim=0)
        m_p = m_p / torch.sum(m_p)
        return torch.sum((m_p-cons_detail)*(m_p-cons_detail))

    def difficulty_loss(self, q, mask):
        mask = mask.unsqueeze_(-1)
        mask = mask.expand(q.shape[0], q.shape[1])
        mask_q = q * mask
        diff_loss = -torch.norm(mask_q, 2)
        penalty_degree = 0.1
        return penalty_degree * diff_loss

    def target_distribution(self, q):
        p = q**2 / torch.sum(q, dim=0)
        p = p / torch.sum(p, dim=1, keepdim=True)
        return p

    def triplet_loss(self, anchor, positive, negative, margin_constant):
        # loss = max(d(anchor, negative) - d(anchor, positve) + margin, 0), margin > 0
        # d(x, y) = q(x) * q(y)
        negative_dis = torch.sum(anchor * negative, dim=1)
        positive_dis = torch.sum(anchor * positive, dim=1)
        margin = margin_constant * torch.ones(negative_dis.shape).cuda()
        diff_dis = negative_dis - positive_dis
        penalty = diff_dis + margin
        triplet_loss = 1*torch.max(penalty, torch.zeros(negative_dis.shape).cuda())

        return torch.mean(triplet_loss)

    def satisfied_constraints(self,ml_ind1,ml_ind2,cl_ind1, cl_ind2,y_pred):
        
        if ml_ind1.size == 0 or ml_ind2.size == 0 or cl_ind1.size == 0 or cl_ind2.size == 0:
            return 1.0

        count = 0
        satisfied = 0
        for (i, j) in zip(ml_ind1, ml_ind2):
            count += 1
            if y_pred[i] == y_pred[j]:
                satisfied += 1
        for (i, j) in zip(cl_ind1, cl_ind2):
            count += 1
            if y_pred[i] != y_pred[j]:
                satisfied += 1

        return float(satisfied)/count


    def predict(self, X, y):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        latent = self.encodeBatch(X)
        q = self.soft_assign(latent)

        # evalute the clustering performance
        y_pred = torch.argmax(q, dim=1).data.cpu().numpy()
        y = y.data.cpu().numpy()
        if y is not None:
            print("acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
            final_acc = acc(y, y_pred)
            final_nmi = normalized_mutual_info_score(y, y_pred)
        return final_acc, final_nmi, y_pred

    def fit(self,anchor, positive, negative, ml_ind1,ml_ind2,cl_ind1, cl_ind2, mask, use_global, ml_p, cl_p, X,y=None, lr=0.001, batch_size=256, num_epochs=10, update_interval=1, tol=1e-3, use_kmeans=True, plotting="",clustering_loss_weight=1):    
        
        # save intermediate results for plotting
        intermediate_results = collections.defaultdict(lambda:{})
        
        '''X: tensor data'''
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        print("=====Training IDEC=======")
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)

        if use_kmeans:
            print("Initializing cluster centers with kmeans.")
            kmeans = KMeans(self.n_clusters, n_init=20)
            data = self.encodeBatch(X)
            y_pred = kmeans.fit_predict(data.data.cpu().numpy())
            y_pred_last = y_pred
            self.mu.data.copy_(torch.Tensor(kmeans.cluster_centers_))
        else:
            # use kmeans to randomly initialize cluster ceters
            print("Randomly initializing cluster centers.")
            kmeans = KMeans(self.n_clusters, n_init=1, max_iter=1)
            data = self.encodeBatch(X)
            y_pred = kmeans.fit_predict(data.data.cpu().numpy())
            y_pred_last = y_pred
            self.mu.data.copy_(torch.Tensor(kmeans.cluster_centers_))

        if y is not None:
            y = y.cpu().numpy()
            # print("Kmeans acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
        self.train()
        num = X.shape[0]
        num_batch = int(math.ceil(1.0*X.shape[0]/batch_size))
        ml_num_batch = int(math.ceil(1.0*ml_ind1.shape[0]/batch_size))
        cl_num_batch = int(math.ceil(1.0*cl_ind1.shape[0]/batch_size))
        tri_num_batch = int(math.ceil(1.0*anchor.shape[0]/batch_size))
        cl_num = cl_ind1.shape[0]
        ml_num = ml_ind1.shape[0]
        tri_num = anchor.shape[0]

        final_acc, final_nmi, final_epoch = 0, 0, 0
        update_ml = 1
        update_cl = 1
        update_triplet = 1
        for epoch in range(num_epochs):
            if epoch%update_interval == 0:
                # update the targe distribution p
                latent = self.encodeBatch(X)
                q = self.soft_assign(latent)
                p = self.target_distribution(q).data

                # evalute the clustering performance
                y_pred = torch.argmax(q, dim=1).data.cpu().numpy()
                if use_global:
                    y_dict = collections.defaultdict(list)
                    ind1, ind2 = [], []
                    for i in range(y_pred.shape[0]):
                        y_dict[y_pred[i]].append(i)
                    for key in y_dict.keys():
                        if y is not None:
                            print("predicted class: ", key, " total: ", len(y_dict[key]))
                            #, " mapped index(ground truth): ", np.bincount(y[y_dict[key]]).argmax())

                if y is not None:
                    print("acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
                    print("satisfied constraints: %.5f"%self.satisfied_constraints(ml_ind1,ml_ind2,cl_ind1, cl_ind2,y_pred))
                    final_acc = acc(y, y_pred)
                    final_nmi = normalized_mutual_info_score(y, y_pred)
                    final_epoch = epoch

                # save model for plotting
                if plotting and (epoch in [10,20,30,40] or epoch%50 == 0 or epoch == num_epochs-1):
                    
                    df = pd.DataFrame(latent.cpu().numpy())
                    df["y"] = y
                    df.to_pickle(os.path.join(plotting,"save_model_%d.pkl"%(epoch)))
                    
                    intermediate_results["acc"][str(epoch)] = acc(y, y_pred)
                    intermediate_results["nmi"][str(epoch)] = normalized_mutual_info_score(y, y_pred)
                    with open(os.path.join(plotting,"intermediate_results.json"), "w") as fp:
                        json.dump(intermediate_results, fp)

                # check stop criterion
                try:
                    delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / num
                    y_pred_last = y_pred
                    if epoch>0 and delta_label < tol:
                        print('delta_label ', delta_label, '< tol ', tol)
                        print("Reach tolerance threshold. Stopping training.")

                        # save model for plotting
                        if plotting:
                            
                            df = pd.DataFrame(latent.cpu().numpy())
                            df["y"] = y
                            df.to_pickle(os.path.join(plotting,"save_model_%d.pkl"%epoch))
                            
                            intermediate_results["acc"][str(epoch)] = acc(y, y_pred)
                            intermediate_results["nmi"][str(epoch)] = normalized_mutual_info_score(y, y_pred)
                            with open(os.path.join(plotting,"intermediate_results.json"), "w") as fp:
                                json.dump(intermediate_results, fp)
                        break
                except:
                    pass

            # train 1 epoch for clustering loss
            train_loss = 0.0
            recon_loss_val = 0.0
            cluster_loss_val = 0.0
            instance_constraints_loss_val = 0.0
            global_loss_val = 0.0
            for batch_idx in range(num_batch):
                xbatch = X[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                pbatch = p[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                mask_batch = mask[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                optimizer.zero_grad()
                inputs = Variable(xbatch)
                target = Variable(pbatch)
                cons_detail = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
                global_cons = torch.from_numpy(cons_detail).float().to("cuda")

                z, qbatch, xrecon = self.forward(inputs)
                if use_global == False:
                    cluster_loss = self.cluster_loss(target, qbatch)
                    recon_loss = self.recon_loss(inputs, xrecon)
                    instance_constraints_loss = self.difficulty_loss(qbatch, mask_batch)
                    loss = cluster_loss + recon_loss + instance_constraints_loss
                    loss.backward()
                    optimizer.step()
                    cluster_loss_val += cluster_loss.data * len(inputs)
                    recon_loss_val += recon_loss.data * len(inputs)
                    instance_constraints_loss_val += instance_constraints_loss.data * len(inputs)
                    train_loss = clustering_loss_weight*cluster_loss_val + recon_loss_val + instance_constraints_loss_val
                else:
                    cluster_loss = self.cluster_loss(target, qbatch)
                    recon_loss = self.recon_loss(inputs, xrecon)
                    global_loss = self.global_size_loss(qbatch, global_cons)
                    loss = cluster_loss + recon_loss + global_loss
                    loss.backward()
                    optimizer.step()
                    cluster_loss_val += cluster_loss.data * len(inputs)
                    recon_loss_val += recon_loss.data * len(inputs)
                    train_loss = clustering_loss_weight*cluster_loss_val + recon_loss_val


            if instance_constraints_loss_val != 0.0:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f Instance Difficulty Loss: %.4f"% (
                    epoch + 1, train_loss / num, cluster_loss_val / num, recon_loss_val / num, instance_constraints_loss_val / num))
            elif global_loss_val != 0.0 and use_global:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f Global Loss: %.4f"% (
                    epoch + 1, train_loss / num + global_loss_val/num_batch, cluster_loss_val / num, recon_loss_val / num, global_loss_val / num_batch))
            else:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f" % (
                    epoch + 1, train_loss / num, cluster_loss_val / num, recon_loss_val / num))
            ml_loss = 0.0
            if epoch % update_ml == 0:
                for ml_batch_idx in range(ml_num_batch):
                    px1 = X[ml_ind1[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    px2 = X[ml_ind2[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    pbatch1 = p[ml_ind1[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[ml_ind2[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    loss = (ml_p*self.pairwise_loss(q1, q2, "ML")+self.recon_loss(inputs1, xr1) + self.recon_loss(inputs2, xr2))
                    # 0.1 for mnist/reuters, 1 for fashion, the parameters are tuned via grid search on validation set
                    ml_loss += loss.data
                    loss.backward()
                    optimizer.step()

            cl_loss = 0.0
            if epoch % update_cl == 0:
                for cl_batch_idx in range(cl_num_batch):
                    px1 = X[cl_ind1[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    px2 = X[cl_ind2[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    pbatch1 = p[cl_ind1[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[cl_ind2[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    loss = cl_p*self.pairwise_loss(q1, q2, "CL")
                    cl_loss += loss.data
                    loss.backward()
                    optimizer.step()

            if ml_num_batch >0 and cl_num_batch > 0:
                print("Pairwise Total:", round(float(ml_loss.cpu()), 2) + float(cl_loss.cpu()), "ML loss", float(ml_loss.cpu()), "CL loss:", float(cl_loss.cpu()))
            triplet_loss = 0.0
            if epoch % update_triplet == 0:
                for tri_batch_idx in range(tri_num_batch):
                    px1 = X[anchor[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    px2 = X[positive[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    px3 = X[negative[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    pbatch1 = p[anchor[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[positive[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    pbatch3 = p[negative[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    inputs3 = Variable(px3)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    target3 = Variable(pbatch3)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    z3, q3, xr3 = self.forward(inputs3)
                    loss = self.triplet_loss(q1, q2, q3, 0.1)
                    triplet_loss += loss.data
                    loss.backward()
                    optimizer.step()
            if tri_num_batch > 0:
                print("Triplet Loss:", triplet_loss)
        return final_acc, final_nmi, final_epoch

In [ ]:
import os
import os.path
import errno
import numpy as np
import gzip
import torch
import pickle
import torch.utils.data as data
import codecs
import urllib
class Netflix(data.Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.use_cuda = torch.cuda.is_available()
        self.training_file = "/content/drive/MyDrive/ECS271/NetFlix Assignment/part1_pred_6000.out" # your data for clustering (e.g., user-predictions matrix, user-embedding learned from SVD ...)
        self.label_file = "/content/drive/MyDrive/ECS271/NetFlix Assignment/clusters_6000.out" # clustering labels (not applicable for our assignment, I used the k-means labels for svd_pu file)
        if download:
            self.download()
        train = np.loadtxt(self.training_file)
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        label = np.loadtxt(self.label_file)
        train = scaler.fit_transform(train)
        if self.train:
            self.train_data, self.train_labels = torch.tensor(train, dtype=torch.float32), torch.tensor(label, dtype=torch.int)
            if self.use_cuda:
                self.train_data = self.train_data.cuda()
                self.train_labels = self.train_labels.cuda()
        else:
            self.test_data, self.test_labels = torch.tensor(train, dtype=torch.float32), torch.tensor(label, dtype=torch.int)
            if self.use_cuda:
                self.test_data = self.test_data.cuda()
                self.test_labels = self.test_labels.cuda()
    def __getitem__(self, index):
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

In [ ]:
#Run_IDEC
netflix_train = Netflix('', train=True)
netflix_test = Netflix('', train=False) # here the test and train are the same
X, y = netflix_train.train_data, netflix_train.train_labels
test_X, test_y = netflix_train.train_data, netflix_train.train_labels

idec = IDEC(input_dim=16142, z_dim=40, n_clusters=140,
           encodeLayer=[500, 500, 2000], decodeLayer=[2000, 500, 500], activation="relu", dropout=0)
# Print network structure
print(idec)

# Set parameters
lr = 0.001
batch_size = 512
epochs = 500
update_interval = 1
ml_penalty, cl_penalty = 0.1, 1
# Construct constraints (here is the baseline so no constraints are provided).
ml_ind1, ml_ind2, cl_ind1, cl_ind2 = np.array([]), np.array([]), np.array([]), np.array([])
anchor, positive, negative = np.array([]), np.array([]), np.array([])
instance_guidance = torch.zeros(X.shape[0]).cuda()
use_global = False
idec.load_model('/content/drive/MyDrive/ECS271/NetFlix Assignment/sdae_netflix_weights.pt') # load your pre-trained model from SDAE.ipynb
# Train the clustering model
train_acc, train_nmi, epo = idec.fit(anchor, positive, negative, ml_ind1, ml_ind2, cl_ind1, cl_ind2, instance_guidance, use_global,  ml_penalty, cl_penalty, X, y,
                          lr=lr, batch_size=batch_size, num_epochs=epochs,update_interval=update_interval,tol=1*1e-4, clustering_loss_weight=1)

#Epoch  25: Total: 0.2302 Clustering Loss: 0.0609 Reconstruction Loss: 0.1692
acc: 0.18967, nmi: 0.45073
satisfied constraints: 1.00000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


#Epoch  26: Total: 0.1886 Clustering Loss: 0.0630 Reconstruction Loss: 0.1256
acc: 0.19678, nmi: 0.45743
satisfied constraints: 1.00000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


#Epoch  27: Total: 0.1474 Clustering Loss: 0.0507 Reconstruction Loss: 0.0967


...

#Epoch 498: Total: 0.0824 Clustering Loss: 0.0289 Reconstruction Loss: 0.0535


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


acc: 0.15631, nmi: 0.42084
satisfied constraints: 1.00000
#Epoch 499: Total: 0.0829 Clustering Loss: 0.0271 Reconstruction Loss: 0.0558
acc: 0.15665, nmi: 0.42057
satisfied constraints: 1.00000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


#Epoch 500: Total: 0.0810 Clustering Loss: 0.0280 Reconstruction Loss: 0.0531


In [ ]:
# Generate predictions on the test data
test_acc, test_nmi, y_pred = idec.predict(test_X, test_y)
np.savetxt("/content/drive/MyDrive/ECS271/NetFlix Assignment/idec_pred.npy", y_pred)

acc: 0.15970, nmi: 0.42054


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
y_pred.shape

(5905,)

In [ ]:
import pandas as pd


In [ ]:
aa=pd.read_csv('/content/drive/MyDrive/ECS271/NetFlix Assignment/train(2).csv')
aa=aa[['movie-id','customer-id','rating']] 


#movie-id to mov# mapping
j=0
mov_map={}
for i in aa['movie-id'].unique():
  mov_map[i]=j 
  j+=1

#customer-id to cus# mapping
j=0
cus_map={}
for i in aa['customer-id'].unique():
  cus_map[i]=j 
  j+=1

users_gt_10=aa['customer-id'].value_counts().reset_index(name="count").query("count > 10")
users_lt_10=aa['customer-id'].value_counts().reset_index(name="count").query("count <= 10") 

user_ids_gt_10=users_gt_10['index'].to_list()
# len(user_ids_gt_10)

dont_choose_from_df=aa[~aa['customer-id'].isin(user_ids_gt_10)] 

users_ids_train=dont_choose_from_df['customer-id'].to_list()
movies_ids_train=dont_choose_from_df['movie-id'].to_list()
users_train=[cus_map[i] for i in users_ids_train]
movies_train=[mov_map[i] for i in movies_ids_train]

X_train_d=list(zip(users_train , movies_train))
Y_train_d=dont_choose_from_df['rating'].to_list() 

choose_from_df=aa.loc[aa['customer-id'].isin(user_ids_gt_10)] 

users_ids=choose_from_df['customer-id'].to_list()
movies_ids=choose_from_df['movie-id'].to_list()
users=[cus_map[i] for i in users_ids]
movies=[mov_map[i] for i in movies_ids]


X_choose=list(zip(users , movies))
Y_choose=choose_from_df['rating'].to_list() 


check=list(zip(users_ids_train,movies_ids_train)) 
check_y=Y_train_d

In [ ]:
labels=y_pred

In [ ]:
from collections import defaultdict
cluster_map=defaultdict(list)
for i in range(len(labels)):
  cluster_map[labels[i]].append(i)

In [ ]:
# total_mean=np.sum(pred)/(95318510)

In [ ]:
pred=np.loadtxt('/content/drive/MyDrive/ECS271/NetFlix Assignment/part1_pred_6000.out') 

In [ ]:
total_mean=np.sum(pred)/(95318510)

In [ ]:
# pred[pred<1]=1
# pred=np.around(pred)
mean_rating_by_user=np.mean(pred,axis=1,keepdims=True)
mean_rating_by_movie=np.mean(pred,axis=0,keepdims=True)
centered_pred=pred - mean_rating_by_user
# centered_pred=pred
# sc=SpectralClustering(
#         n_clusters=110,
#         eigen_solver="arpack",
#         affinity='nearest_neighbors'
#     )
# sc.fit(pred)
# sc = SpectralClustering(n_clusters=27).fit(pred)
# print(sc)
# labels = sc.labels_

In [ ]:
from numpy import dot
from numpy.linalg import norm



def predict_from_sc(user_id,mov_id):
  #handle cases if customer and movie not seen before
  if user_id not in cus_map and mov_id not in mov_map:
    return total_mean

  # handle cases if customer not seen before
  if user_id in cus_map and mov_id not in mov_map:
    return round(mean_rating_by_user[cus_map[user_id]][0])

  #handle cases if movie not seen before
  if user_id not in cus_map and mov_id in mov_map:
    return round(mean_rating_by_movie[0][mov_map[mov_id]])  
  
  
  #user_id and mov_id not converted to indices in matrix yet.
  cus=cus_map[user_id]
  mov=mov_map[mov_id]
  
  
  c=labels[cus]
  mm=centered_pred[cus]
  #other users in the same cluster
  cus_in_cluster=cluster_map[c]
  wt=[]
  r=[]
  for i in cus_in_cluster:
    mi=centered_pred[i]
    w=np.dot(mm,mi)/(norm(mm)*norm(mi))
    wt.append(w)
    r.append(round(pred[i][mov])) 
  ss=0
  for i in range(len(wt)):
    ss+=wt[i]*r[i]
  # print('ss: ',ss)
  # print('sum wt: ',sum(wt))
  ans=ss/sum(wt)
  ans=round(ans)
  # ans=pred[cus][mov]
  # ans=round(ans)
  ans=min(5,ans)
  ans=max(0,ans)
  return ans   




In [ ]:
s=0
ans_y=[]
for i in range(len(check)):
  a,b=check[i]
  y=predict_from_sc(a,b)
  ans_y.append(y)
  s+=(check_y[i]-y)**2 
print(s/len(check))  

1.1643238002908385


**Predicting on the test.csv**

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ECS271/NetFlix Assignment/test.csv',header=None)

In [ ]:
test.columns=['movie-id','customer-id','rating','date']
test.head()

,movie-id,customer-id,rating,date
0,11089,11765,?,2005-06-03
1,11279,5858,?,2005-05-07
2,12140,5202,?,2004-01-24
3,12191,18818,?,2005-03-27
4,12299,10021,?,2005-09-07


In [ ]:
c=test['customer-id'].to_list()
m=test['movie-id'].to_list()
r=test['rating'].to_list()

In [ ]:
from google.colab import output 
ans_1=[]
for i in range(len(c)):
  print(i)
  if i%10==0: 
    output.clear()
  aa=predict_from_sc(c[i],m[i])  
  ans_1.append(aa)


249991
249992
249993
249994
249995
249996
249997
249998
249999


In [ ]:
total_mean

3.6107077275916444

In [ ]:
ans_1=np.array(ans_1)

In [ ]:
ans_1=np.clip(ans_1,1,5)

In [ ]:
np.max(ans_1)

5

In [ ]:
np.save('/content/drive/MyDrive/ECS271/NetFlix Assignment/ans_1_p2.npy',ans_1)